<a href="https://colab.research.google.com/github/Jacquesjh/MnistClassifier/blob/main/MNIST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%cd /content/drive/MyDrive/Machine Learning/Mnist

/content/drive/MyDrive/Machine Learning/Mnist


#Vamos começar importando o Mnist dataset, e então iremos pré processar seus dados, preparando-os para nossa rede neural

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib as mpl
import tensorflow_datasets as tfds

###Fazendo o download do nosso dataset

In [4]:
##O dataset do Mnist consiste de 70,000 imagens de digitos escritos à mão, de dimensões 28x28 pixel
##Ele já vem dividido entre training set e testing set, contendo 60,000 e 10,000 imagens, respectivamente
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True, as_supervised = True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [5]:
##Aqui podemos ter algumas informações sobre como o mnist_dataset é composto
print(mnist_info.splits)
print(mnist_info.features)

{'test': <tfds.core.SplitInfo num_examples=10000>, 'train': <tfds.core.SplitInfo num_examples=60000>}
FeaturesDict({
    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})


In [6]:
##Dividindo o dataset inteiro entre dataset de treino e de test
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [7]:
mnist_train.element_spec

(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

##Com um entendimento melhor sobre nosso dataset, podemos começar a tratá-lo

##Algo importante para nosso pré processamento, é escalonar os valores dos dados
###Os valores padrões estão numa escala de cinza de 0 a 255, porém, é interessante mudar seu domínio para valores entre 0 e 1

In [8]:
##Aqui nos escalonamos os valores de cada dado que está entre 0 e 255 para um float entre 0 e 1
def scale(image, label):

  image = tf.cast(image, tf.float32)
  image = image/255.0

  return image, label

In [9]:
##E aplicamos esta transformação nos nossos dados
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

##Agora podemos criar nosso set de validation e embaralhar nossos dados

In [10]:
##Separando o número de dados para nosso set de validation
num_validation_samples = 0.1666667 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [11]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [12]:
BUFFER_SIZE = 10000
shuffled_train_val_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_val_data.take(num_validation_samples)
train_data = shuffled_train_val_data.skip(num_validation_samples)

BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

#Assim, podemos começar a modelar a nossa rede neural

In [13]:
##Criando a arquitetura da nossa rede neural
model = tf.keras.Sequential([
                             tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', input_shape = (28, 28, 1)),
                             tf.keras.layers.MaxPooling2D((2, 2)),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(64, activation = 'relu'),
                             tf.keras.layers.Dense(10, activation = 'softmax')
])

In [14]:
##Aqui podemos visualizar sua estrutura
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 10816)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                692288    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 693,578
Trainable params: 693,578
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

##Então, chegou a hora de treinar nossa rede nos nossos dados

In [17]:
max_epochs = 10

call_back = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 2)

model.fit(
          train_data,
          batch_size = 128,
          epochs = max_epochs,
          validation_data = (validation_inputs, validation_targets),
          verbose = 2
)

Epoch 1/10
500/500 - 5s - loss: 0.1901 - accuracy: 0.9402 - val_loss: 0.0839 - val_accuracy: 0.9755
Epoch 2/10
500/500 - 3s - loss: 0.0633 - accuracy: 0.9807 - val_loss: 0.0547 - val_accuracy: 0.9831
Epoch 3/10
500/500 - 3s - loss: 0.0392 - accuracy: 0.9882 - val_loss: 0.0426 - val_accuracy: 0.9882
Epoch 4/10
500/500 - 3s - loss: 0.0287 - accuracy: 0.9909 - val_loss: 0.0385 - val_accuracy: 0.9892
Epoch 5/10
500/500 - 3s - loss: 0.0204 - accuracy: 0.9936 - val_loss: 0.0333 - val_accuracy: 0.9904
Epoch 6/10
500/500 - 3s - loss: 0.0156 - accuracy: 0.9954 - val_loss: 0.0319 - val_accuracy: 0.9909
Epoch 7/10
500/500 - 3s - loss: 0.0134 - accuracy: 0.9960 - val_loss: 0.0237 - val_accuracy: 0.9939
Epoch 8/10
500/500 - 3s - loss: 0.0120 - accuracy: 0.9965 - val_loss: 0.0253 - val_accuracy: 0.9935
Epoch 9/10
500/500 - 3s - loss: 0.0084 - accuracy: 0.9977 - val_loss: 0.0193 - val_accuracy: 0.9953
Epoch 10/10
500/500 - 3s - loss: 0.0065 - accuracy: 0.9981 - val_loss: 0.0163 - val_accuracy: 0.9961

##Com isso, podemos testar nossa rede treinada com o nosso set de treino, para verficar a real precisão dela

In [23]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 2ms/step - loss: 0.0452 - accuracy: 0.9878


In [24]:
print('A precisão da nossa rede neural é de: {0:.2f}%.\nE a nossa perda é de: {1:.2f}'.format(test_accuracy*100, test_loss))

A precisão da nossa rede neural é de: 98.78%.
E a nossa perda é de: 0.05
